In [2]:
from cloudvolume import CloudVolume, Skeleton
import numpy as np

In [3]:
vol_im = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/ch1_iso")
vol_axons = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/axons")

In [ ]:
vol_im.viewer()

In [ ]:
from cloudvolume.exceptions import SkeletonDecodeError
for i in range(66):
    try:
        verts = vol_axons.skeleton.get(i).vertices
    except SkeletonDecodeError:
        continue
    points = np.array(np.divide(verts, vol_axons.resolution))
    points = [list(c) for c in points]

    print(f"ID: {i} w/{len(points)} points")
    if len(points) <= 10000:
        with open("/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/target_space/" + str(i) + ".txt", "w") as f:
            for point in points:
                f.write(f"({point[0]},{point[1]},{point[2]})")
                f.write("\n")

In [ ]:
from cloudreg.scripts.transform_points import NGLink
from cloudreg.scripts.visualization import create_viz_link_from_json

viz_link_init = "https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=7m7rEoVJdsNt1Q"

viz = NGLink(viz_link_init.split("json_url=")[-1])
ngl_json = viz._json
ngl_json['layers'].append(
    {
        "type": "annotation",
        "annotations": points,
        "name" : "test_points"
    }
)
viz_link = create_viz_link_from_json(ngl_json)
print(viz_link)

python -m cloudreg.scripts.transform_points --target_viz_link https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=x4nz2KzMfzinTQ --atlas_viz_link https://ara.viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=HvyNDGaPsd1wyg --affine_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_A.mat --velocity_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_v.mat --soma_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/path_coords.txt

### Write new skeleton layer

Copy and paste info files 

In [4]:
vol_axons_atlas = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/axons_atlas", compress=False)
vol_axons_atlas.chunk_size

Vec(128,128,128, dtype=int64)

### Read transformed points

In [29]:
from os import listdir
from os.path import isfile, join
import json

dir = '/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/atlas_space'
names = [f for f in listdir(dir) if isfile(join(dir, f))]
nums = [int(n.split('points')[-1][:-5]) for n in names]
files = [join(dir,f) for f in names]

for num,file in zip(nums, files):
    skel_targ = vol_axons.skeleton.get(num)
    with open(file) as f:
        json_file = json.load(f)
        coords = []
        for point in json_file:
            new_coord = np.multiply(point['point'], vol_axons_atlas.resolution)
            coords.append(new_coord)
        
        coords = np.array(coords, dtype='float32')
    
    skel_atlas = Skeleton(segid = num, vertices=coords, edges=skel_targ.edges, space='physical')
    skel_atlas.extra_attributes = [ attr for attr in skel_atlas.extra_attributes if attr['id'] == 'radius' ]
    vol_axons_atlas.skeleton.upload(skel_atlas)
    #vol_axons_atlas.skeleton.upload_raw(num, skel_atlas.vertices, skel_atlas.edges, skel_atlas.radii, skel_atlas.vertex_types)

vol_axons_atlas.skeleton.meta.commit_info()  
    

In [5]:
vol_axons_atlas.viewer()

Neuroglancer server listening to http://localhost:1337


127.0.0.1 - - [12/May/2022 14:22:49] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2022 14:22:49] "GET /skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2022 14:22:51] code 404, message /10000.0_10000.0_10000.0/768-896_256-384_768-896: Not Found
127.0.0.1 - - [12/May/2022 14:22:51] "GET /10000.0_10000.0_10000.0/768-896_256-384_768-896 HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2022 14:22:51] code 404, message /10000.0_10000.0_10000.0/768-896_256-384_640-768: Not Found
127.0.0.1 - - [12/May/2022 14:22:51] "GET /10000.0_10000.0_10000.0/768-896_256-384_640-768 HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2022 14:22:51] code 404, message /10000.0_10000.0_10000.0/640-768_256-384_768-896: Not Found
127.0.0.1 - - [12/May/2022 14:22:51] "GET /10000.0_10000.0_10000.0/640-768_256-384_768-896 HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2022 14:22:51] code 404, message /10000.0_10000.0_10000.0/768-896_384-512_640-768: Not Found
127.0.0.1 - - [12/May/2022 14:22:51] "GET /10000.0_10000.0_10000.0/768-896_384-51


Continuing program execution...


In [ ]:
skel = vol_axons.skeleton.get(1)
print(skel)
print(type(skel.vertices))
print(skel.vertices.dtype)
print(np.divide(skel.vertices, vol_axons.resolution))

In [ ]:
np.divide(vol_axons.skeleton.get(1).vertices, [10000,10000,10000])

In [ ]:
np.divide(vol_axons.skeleton.get(1).vertices, vol_axons.resolution)